In [1]:
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et
print('ready')

tree = et.ElementTree(file="xmlfiles/2018kane.xml")
root = tree.getroot()
print(root.tag, root.attrib)


# Let's get all the choices
chcols = ['contestID','choiceID','type','name','party','dispOrder']
df_choices = pd.DataFrame(columns=chcols)

for elem in root.findall(".//County/Election/Contests/Contest"):
    contestID = elem.attrib.get('id')
    for ch in elem.findall('./Choice'):
        choiceID = ch.attrib.get('id')
        chtype = ch.attrib.get('type')
        chname = ch.attrib.get('name')
        dispOrder = ch.attrib.get('displayOrder')
        party = ch.attrib.get('party')
        df_choices = df_choices.append( pd.Series( [contestID, choiceID, chtype, chname, party, dispOrder], index=chcols) ,ignore_index=True)

df_choices['cont_ch'] = df_choices['contestID'] + '_' + df_choices['choiceID']

df_choices.to_csv('csv/df_choices3.csv', index=False, encoding="utf-8")
print('df_choices done')

ready
EDX {'creationDateTime': '2018-03-15T09:17:29'}
df_choices done


In [7]:
data = []

for elem in root.findall(".//County/Election/ElectionTally/PrecinctTally/SplitTally/ContestTally"):
    contestID = elem.attrib.get('contest')
    for choice in elem.findall('./CastVoteTally'):
        choiceID = choice.attrib.get('choice')
        count = choice.attrib.get('count')
        data.append(( contestID, choiceID, count ))

df = pd.DataFrame(data ,columns=['contestID', 'choiceID','count'] )
df.head(5)

,contestID,choiceID,count
0,182,110,0
1,182,115,0
2,182,120,0
3,182,130,0
4,182,135,0


In [8]:
df.shape

(67491, 3)

In [9]:
# Let's get vote counts for every contest in every split
# finally, let's make sure the count column is an int,
# then do a pivot table to sum count based on cont_ch


contest = ['contestID']
df_contest = pd.DataFrame(columns=contest)

for elem in root.findall(".//County/Election/ElectionTally/PrecinctTally/SplitTally/ContestTally"):
    contestID = elem.attrib.get('contest')
    print(contestID)
    df_contest = df_contest.append( pd.Series( [contestID], index=contest) ,ignore_index=True)

df_contest.head(5)

182
192
202
212
222
352
670
671
1202
2202
3007
3032
3037
7370
4022
4452
9000
8006
8014
181
191
201
211
221
351
1201
2201
3006
3031
3036
7369
4021
4451
9001
8006
8014
8006
8014
352
351
182
192
202
212
222
352
670
671
1872
3007
3032
3037
7370
4022
9002
8006
8014
181
191
201
211
221
351
1871
3006
3031
3036
7369
4021
9003
8006
8014
8006
8014
352
351
182
192
202
212
222
352
670
671
1202
2202
3007
3032
3037
7370
4022
4452
9004
8006
8014
181
191
201
211
221
351
1201
2201
3006
3031
3036
7369
4021
4451
9005
8006
8014
8006
8014
352
351
182
192
202
212
222
352
670
671
1202
2202
3007
3032
3037
7370
4022
9004
8006
8014
181
191
201
211
221
351
1201
2201
3006
3031
3036
7369
4021
9005
8006
8014
8006
8014
182
192
202
212
222
352
670
671
1872
3007
3032
3037
7370
4022
4452
9006
8006
8014
181
191
201
211
221
351
1871
3006
3031
3036
7369
4021
4451
9007
8006
8014
8006
8014
352
351
182
192
202
212
222
352
670
671
1202
2212
3007
3032
3037
7370
4022
4452
9008
8006
8014
181
191
201
211
221
351
1201
2211
3006
30

,contestID
0,182
1,192
2,202
3,212
4,222


In [17]:

tally = ['contestID','choiceID','count']
df_tally = pd.DataFrame(columns=tally)

for elem in root.findall(".//County/Election/ElectionTally/PrecinctTally/SplitTally/ContestTally"):
    contestID = elem.attrib.get('contest')
    for choice in elem.findall('./CastVoteTally'):
        choiceID = choice.attrib.get('choice')
        count = choice.attrib.get('count')
        df_tally = df_tally.append( pd.Series( [contestID, choiceID, count], index=tally) ,ignore_index=True)

df_tally['cont_ch'] = df_tally['contestID'] + '_' + df_tally['choiceID']

df_tally.head(5)

,contestID,choiceID,count,cont_ch
0,182,110,0,182_110
1,182,115,0,182_115
2,182,120,0,182_120
3,182,130,0,182_130
4,182,135,0,182_135


In [18]:
df_tally.to_csv('csv/df_test.csv')

In [ ]:
df_tally['count'] = (df_tally['count']).astype(int)
df_tally = pd.pivot_table(df_tally, index=['cont_ch'], aggfunc=np.sum)
df_tally.to_csv('csv/df_sum3.csv')
print('df_sum done')